## Setup and Data Loading
First, we import the necessary libraries and mount Google Drive to access the data files stored there. This setup is crucial for loading the velocity data, which we will analyze.

In [ ]:
import numpy as np
from scipy.integrate import tplquad, dblquad
from google.colab import drive
import math

# Mount Google Drive to access the data file
drive.mount('/content/drive')
# Load velocity data from a numpy file stored on Google Drive
vel = np.load('/content/drive/MyDrive/Colab Notebooks/dmvposnp.npy')

## Constants and Initial Data Preparation
Define constants used throughout the notebook and prepare data for further processing. `AtomicMassSi` is the atomic mass of silicon in atomic mass units (amu), and `bin_number` defines how finely we bin our data in histograms.

In [ ]:
AtomicMassSi = 28.085  # Atomic mass of silicon in amu
bin_number = 10  # Number of bins for the histogram

# Calculate ranges for each component and create bins
ranges = [(vel[:, i].min(), vel[:, i].max()) for i in range(3)]
bins = [np.linspace(ranges[i][0], ranges[i][1], bin_number) for i in range(3)]
hist, edges = np.histogramdd(vel[:, :3], bins=bins, density=True)

# Calculate bin widths based on the ranges
dx = (ranges[0][1] - ranges[0][0]) / bin_number
dy = (ranges[1][1] - ranges[1][0]) / bin_number
dz = (ranges[2][1] - ranges[2][0]) / bin_number

## Helper Functions
Here we define helper functions to process the histogram data and perform transformations.

In [ ]:
def vel_to_bin(v):
    return tuple(math.floor((v[i] - ranges[i][0]) / (ranges[i][1] - ranges[i][0]) * (bin_number - 1)) for i in range(3))

def delta(x, eps):
    return 1.0 / (eps * np.sqrt(2 * np.pi)) * np.exp(-x**2 / (2 * eps**2))

## Radon Transform Implementation
The Radon transform $R[f](\theta, s)$ of a 2D function $f(x, y)$ is given by:

\begin{equation}
R[f](\theta, s) = \int_{-\infty}^{\infty} f(x', y') \delta(x' \cos\theta + y' \sin\theta - s) \, dx' \, dy',
\end{equation}
where:
- $\theta$: angle of projection (in radians).
- $s$: distance from the origin along the projection direction.
- $\delta$: Dirac delta function.

Implement the loop-based Radon transform for integration over the histogram.

In [ ]:
def LoopRadontransform(w, wx, wy, wz):
    integral = 0.0
    for i in range(bin_number):
        for j in range(bin_number):
            for k in range(bin_number):
                x_val = ranges[0][0] + (i + 0.5) * dx
                y_val = ranges[1][0] + (j + 0.5) * dy
                z_val = ranges[2][0] + (k + 0.5) * dz
                delta_val = delta(wx * x_val + wy * y_val + wz * z_val - w, 25.)
                integral += delta_val * hist[i, j, k]
    return integral

## Physical Model Functions
Define rate functions that calculate the effects based on physical models, using the Radon transform.

In [ ]:
def FhatG(l, b, e, m, AA):
    c = 299792.458  # speed of light in km/s
    M = AA * 0.93149  # convert atomic mass unit to GeV
    Mu = M * m / (M + m)
    w = np.abs(M * e / Mu) * np.sqrt(2 * M * ((10**6) / (c**2)) * e)
    nxg = np.cos(b) * np.cos(l)
    nyg = np.cos(b) * np.sin(l)
    nzg = np.sin(b)
    return LoopRadontransform(w, nxg, nyg, nzg)

## Rate Calculation and Mapping
Use the above-defined physical models to calculate rates based on galactic coordinates and create a map.

In [ ]:
def xy2lb(x, y):
    theta = np.arcsin(y / np.sqrt(2))
    l = np.pi * (-x) / (2.0 * np.sqrt(2.0) * np.cos(theta))
    b = np.arcsin((2.0 * theta + np.sin(2.0 * theta)) / np.pi)
    return l, b

def rate_mollweide(x, y):
    l, b = xy2lb(x, y)
    return FhatG(l, b, 5.0, 100.0, AtomicMassSi)  # Example parameters for energy and mass

xx = np.linspace(-2.0 * np.sqrt(2), 2.0 * np.sqrt(2), 10)
yy = np.linspace(-np.sqrt(2), np.sqrt(2), 10)
map_data = [rate_mollweide(x, y) for x in xx for y in yy if (x**2 / 4 + y**2) < 2]

np.save('/content/drive/MyDrive/Colab Notebooks/mapsdata.npy', map_data)

## Mollweide Mapping
The Mollweide projection is used to represent the celestial sphere onto a 2D map. It is particularly useful in astronomy for visualizing large-scale data across the entire sky.

The transformation from latitude $\phi$ and longitude $\lambda$ to coordinates $(x, y)$ is defined as:

\begin{equation}
x = 2 \sqrt{2} \frac{\lambda \cos\theta}{\pi}, \quad y = \sqrt{2} \sin\theta,
\end{equation}
where $\theta$ is an auxiliary variable satisfying:

\begin{equation}
2\theta + \sin(2\theta) = \pi \sin\phi.
\end{equation}
Here, $\theta$ is solved numerically.

## Event Rate Equations
The event rate $F(\ell, b, E)$ in terms of galactic coordinates $(\ell, b)$ and energy $E$ is given by:

\begin{equation}
F(\ell, b, E) = \int \rho(v) \cdot \sigma(v, E) \cdot v \, dv,
\end{equation}
where:
- $\rho(v)$: velocity distribution.
- $\sigma(v, E)$: cross-section depending on velocity and energy.
- $v$: velocity magnitude.

In the code, this is implemented using the Radon transform to project the velocity distribution onto specific directions defined by $\ell$ and $b$.